In [3]:
import pandas as pd
import numpy as np

In [97]:
#Import data

customer_content = "customer_content.csv"
data_ticket = "data_ticket.csv"
working_hours = "working_hours.csv"


df_customer_content = pd.read_csv(data_ticket)
df_data_ticket = pd.read_csv(data_ticket)
df_working_hours = pd.read_csv(working_hours)

In [77]:
df_working_hours.head(100)

,AUTO_ID,AGENT_ID,TYPE_ACTION,CREATE_DATE
0,228297,bpomb02,START_SHIFT,1748742713000
1,228301,bpomb02,START_BREAK,1748742718000
2,228302,bpomb02,START_WORKING,1748742720000
3,228303,bpomb02,START_BREAK,1748742731000
4,228304,bpomb02,START_WORKING,1748742740000
...,...,...,...,...
95,229787,bpomb02,START_WORKING,1748845008000
96,229758,bpomb02,START_SHIFT,1748846376000
97,229788,bpomb02,START_BREAK,1748846401000
98,229759,bpomb02,END_SHIFT,1748848862000


In [78]:
df_working_hours["event_time_vn"] = (
    pd.to_datetime(df_working_hours["CREATE_DATE"], unit="ms", utc=True)
      .dt.tz_convert("Asia/Ho_Chi_Minh")
)


In [83]:
# 6. Định nghĩa các cặp hợp lệ (UPDATED)
valid_pairs = {
    ("START_SHIFT", "START_WORKING"),
    ("START_WORKING", "START_BREAK"),
    ("START_BREAK", "START_WORKING"),
    ("START_WORKING", "END_SHIFT"),
}

# 7. Check previous → current (chấp nhận NULL)
valid_prev = (
    df_working_hours["prev_event"].isna() |
    pd.Series(
        list(zip(
            df_working_hours["prev_event"],
            df_working_hours["TYPE_ACTION"]
        )),
        index=df_working_hours.index
    ).isin(valid_pairs)
)

# 8. Check current → next (chấp nhận NULL)
valid_next = (
    df_working_hours["next_event"].isna() |
    pd.Series(
        list(zip(
            df_working_hours["TYPE_ACTION"],
            df_working_hours["next_event"]
        )),
        index=df_working_hours.index
    ).isin(valid_pairs)
)

# 9. Final status
df_working_hours["event_status"] = (
    (valid_prev & valid_next)
        .map({True: "TRUE", False: "FAIL"})
)


In [84]:
df_working_hours.to_csv('test.csv')

### Câu 3

In [111]:
df_data_ticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60675 entries, 0 to 60674
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             60675 non-null  datetime64[ns]
 1   Agent_tiep_nhan  60675 non-null  object        
 2   Level1           60675 non-null  object        
 3   Total_ticket     60675 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.9+ MB


In [101]:
df_data_ticket['Agent_tiep_nhan'] = df_data_ticket['Agent_tiep_nhan'].fillna('XNA')
df_data_ticket['Level1'] = df_data_ticket['Level1'].fillna('XNA')

In [109]:
df_data_ticket=df_data_ticket.rename(columns={
                                     'Date ':'Date'
                                    })

In [113]:
df_data_ticket['Date']=pd.to_datetime(df_data_ticket['Date'])
df_data_ticket['Week']=df_data_ticket['Date'].dt.isocalendar().week
df_data_ticket['Month']=df_data_ticket['Date'].dt.month
df_data_ticket['Year']=df_data_ticket['Date'].dt.year

In [118]:
total_ticket_per_week=df_data_ticket.groupby(['Year','Month','Week']).agg({
    'Date':'min','Total_ticket':'sum'
}).reset_index()

In [119]:
total_ticket_per_week

,Year,Month,Week,Date,Total_ticket
0,2025,6,22,2025-06-01,1940
1,2025,6,23,2025-06-02,17998
2,2025,6,24,2025-06-09,17311
3,2025,6,25,2025-06-16,19055
4,2025,6,26,2025-06-23,19422
5,2025,6,27,2025-06-30,3084
6,2025,7,27,2025-07-01,16871
7,2025,7,28,2025-07-07,20585
8,2025,7,29,2025-07-14,21457
9,2025,7,30,2025-07-21,22374


#### 4.  Identify the most frequent issue reported by customers? 

In [197]:
issue_group_map = {

    # ===================== ACC | Account / Security =====================
    "Hỗ trợ tài khoản": "ACC",
    "Xác thực tài khoản": "ACC",
    "Xác thực tài khoản (eKYC)": "ACC",
    "Mở tài khoản": "ACC",
    "Tài khoản Account": "ACC",
    "Tài Khoản": "ACC",
    "Mật khẩu": "ACC",
    "Khóa Ví": "ACC",
    "Mở khóa Ví": "ACC",
    "Khóa tài khoản": "ACC",
    "Hạn mức Ví": "ACC",
    "Hạn mức 24h": "ACC",
    "Ví Trả Sau": "ACC",
    "Nguồn tiền hạn mức cao": "ACC",
    "Điểm tin cậy": "ACC",
    "Chính sách riêng tư": "ACC",
    "Liên kết": "ACC",
    "Liên kết tài khoản": "ACC",
    "Liên kết ngân hàng": "ACC",
    "Liên kết ngân hàng 1 chạm": "ACC",
    "Hủy liên kết": "ACC",
    "Mở chặn giao dịch": "ACC",
    "Xem số dư Ngân hàng": "ACC",

    # ===================== PAY | Transfer / Topup / Withdraw =====================
    "Nạp tiền": "PAY",
    "Ngân hàng - Nạp tiền": "PAY",
    "Nạp tiền từ Ngân hàng vào Ví": "PAY",
    "Nạp tiền điện thoại": "PAY",
    "Nạp data": "PAY",
    "Rút tiền": "PAY",
    "Rút tiền từ Ví về Ngân hàng": "PAY",
    "Rút tiền về NHLK": "PAY",
    "Chuyển tiền đến Ngân hàng": "PAY",
    "Chuyển/nhận tiền Ví - Ví": "PAY",
    "Chuyển/nhận tiền Ví - Ví Nonbank": "PAY",
    "Chuyển tiền đến Ví MoMo": "PAY",
    "Chuyển nhận tiền Ví MoMo": "PAY",
    "Chuyển/nhận tiền qua Chat": "PAY",
    "Chuyển nhận tiền qua chat": "PAY",
    "Chuyển Nhận Tiền": "PAY",
    "Kiều hối": "PAY",
    "Link nhận tiền": "PAY",
    "Nhắc trả tiền": "PAY",
    "Chia tiền": "PAY",
    "Quỹ nhóm": "PAY",

    # ===================== BILL | Bill / Utility =====================
    "Điện": "BILL",
    "Nước": "BILL",
    "Internet": "BILL",
    "Truyền hình": "BILL",
    "Truyền hình trực tuyến": "BILL",
    "Thanh toán online": "BILL",
    "Thanh toán online - Top Brand": "BILL",
    "Thanh toán trực tiếp": "BILL",
    "Thanh toán tại quầy (Offline)": "BILL",
    "Thanh toán điện thoại trả sau": "BILL",
    "Thanh toán điện thoại cố định": "BILL",
    "Thanh toán trung gian Viễn thông Tiện ích": "BILL",
    "Thu hộ bảo hiểm y tế": "BILL",
    "Thu hộ Bảo hiểm": "BILL",
    "Thu hộ thẻ tín dụng": "BILL",
    "Viện phí": "BILL",
    "Học phí": "BILL",
    "Chung cư": "BILL",
    "Dịch vụ công": "BILL",
    "Viễn thông tiện ích": "BILL",
    "Xăng dầu": "BILL",
    "Xuất hóa đơn": "BILL",
    "Dịch vụ VTTI": "BILL",
    "Dịch vụ Offline payments": "BILL",

    # ===================== MER | Merchant / Business =====================
    "QR đa năng": "MER",
    "QR Đa năng dùng MoMo quét": "MER",
    "QR Đa năng dùng App Bank/ Ví điện tử khác quét": "MER",
    "QR Bán hàng dùng MoMo quét": "MER",
    "QR Bán hàng dùng App Bank/ Ví điện tử khác quét": "MER",
    "QR bán hàng": "MER",
    "Merchant Care": "MER",
    "Doanh nghiệp/Chuỗi cửa hàng": "MER",
    "Trang doanh nghiệp": "MER",
    "EPS - ENTERPRISE PAYMENT & BUSINESS SOLUTIONS": "MER",
    "Dịch vụ W2W/W2B/Merchant Care": "MER",
    "W2B - Chuyển tiền tới ngân hàng": "MER",
    "W2B - AS - Chuyển tiền tới ngân hàng": "MER",
    "Quảng cáo": "MER",
    "Truy thu theo yêu cầu từ đối tác": "MER",
    "Quản lý thẻ thành viên": "MER",
    "Dịch vụ Marketing & Distribution": "MER",
    "Dịch vụ Digital platforms": "MER",
    "Ngân hàng - Liên kết": "MER",

    # ===================== FIN | Financial Services =====================
    "Vay Nhanh": "FIN",
    "Chi hộ vay tiêu dùng": "FIN",
    "Thu hộ vay tiêu dùng": "FIN",
    "Chi hộ vay kinh doanh": "FIN",
    "Tài chính siêu tốc": "FIN",
    "Đầu tư tích lũy": "FIN",
    "Túi thần tài": "FIN",
    "Heo đất MoMo": "FIN",
    "Chứng khoán": "FIN",
    "Chứng chỉ quỹ": "FIN",
    "Mua bảo hiểm": "FIN",
    "Thanh toán phí bảo hiểm": "FIN",
    "Tài Chính - Bảo Hiểm": "FIN",
    "Trung tâm tài chính": "FIN",
    "Quản lý chi tiêu": "FIN",
    "Vượt Ngàn Chi Tiêu": "FIN",
    "Quốc tế": "FIN",
    "Xổ số": "FIN",
    "Mở thẻ tín dụng": "FIN",
    "Tính năng tài chính": "FIN",
    "Dịch vụ FS": "FIN",

    # ===================== COM | Commerce / Lifestyle =====================
    "Thương mại điện tử": "COM",
    "Sàn thương mại điện tử": "COM",
    "Mua sắm hoàn tiền": "COM",
    "Thẻ quà tặng": "COM",
    "Mua mã thẻ di động": "COM",
    "Mua mã thẻ data": "COM",
    "Thanh toán game/ mua mã thẻ game": "COM",
    "Gói combo nhà mạng": "COM",
    "Combo Data Flexible": "COM",
    "Vé máy bay": "COM",
    "Vé tàu hỏa": "COM",
    "Vé xe khách": "COM",
    "Vé xem phim": "COM",
    "Du lịch": "COM",
    "Khách sạn": "COM",
    "Mua sim": "COM",
    "Nội dung số": "COM",
    "Giải trí": "COM",
    "Thời trang": "COM",
    "Trả góp sản phẩm Apple": "COM",

    # ===================== PRO | Promotion / Rewards =====================
    "MoMo Rewards": "PRO",
    "MoMo Reward": "PRO",
    "Khuyến mãi hàng ngày": "PRO",
    "Chương Trình Khuyến Mãi": "PRO",
    "Chương trình khuyến mãi - Nội bộ": "PRO",
    "Giới thiệu bạn bè": "PRO",
    "Lì xì": "PRO",
    "Một Vòng MoMo – Quay 100% trúng, tổng thưởng đến 3 tỷ": "PRO",
    "Vòng Quay \"Chill\" Tài": "PRO",
    "Vòng Phát Tài": "PRO",
    "Đấu Trường Tri Thức": "PRO",
    "Sống tốt cùng MoMo": "PRO",
    "Thổ địa MoMo": "PRO",
    "Đi bộ cùng MoMo": "PRO",
    "Tích tem dễ dàng, săn ngàn ưu đãi từ thương hiệu Nhật và quà Conan độc quyền": "PRO",

    # ===================== APP | App / Platform =====================
    "Hướng dẫn sử dụng ứng dụng": "APP",
    "Các tính năng trên MoMo": "APP",
    "Tính năng": "APP",
    "Tính Năng": "APP",
    "Mini Apps": "APP",
    "Báo lỗi ứng dụng": "APP",
    "Lỗi liên quan nền tảng": "APP",
    "Nền tảng ứng dụng": "APP",
    "Màn hình Kết quả giao dịch": "APP",
    "Màn hình thanh toán": "APP",
    "Màn hình thanh toán an toàn": "APP",
    "Trò chuyện trên ứng dụng": "APP",
    "Trang cá nhân trên ứng dụng": "APP",
    "Hộ chiếu sinh viên": "APP",
    "XNA": "APP",
    "Môi trường": "APP",
    "Đối tác MoMo": "APP",
    "Điểm hỗ trợ MoMo": "APP",

    # ===================== CS | Customer Support =====================
    "CSKH": "CS",
    "Giải quyết khiếu nại": "CS",
    "Phản ánh lừa đảo": "CS",
    "Góp ý": "CS",
    "Góp ý chưa rõ ràng": "CS",
    "Chưa xác định được vấn đề": "CS",

    # ===================== OTHER | Long-tail =====================
    "Khác": "OTHER",
    "Dịch vụ khác": "OTHER"
}

df_data_ticket["issue_group_code"] = (
    df_data_ticket["Level1"]
    .map(issue_group_map)
)


In [198]:
count_per_group=df_data_ticket.groupby('issue_group_code').agg({'Total_ticket':'sum'}).reset_index().sort_values('Total_ticket', ascending=False)

In [199]:
count_per_group

,issue_group_code,Total_ticket
5,FIN,53978
4,CS,49629
0,ACC,18400
8,PAY,14422
6,MER,12304
3,COM,11558
2,BILL,8361
1,APP,7231
9,PRO,1634
7,OTHER,232


In [200]:
count_per_issue=df_data_ticket.groupby(['issue_group_code','Level1']).agg({'Total_ticket':'sum'}).reset_index().sort_values('Total_ticket', ascending=False)

In [202]:
count_per_issue.head(10)

,issue_group_code,Level1,Total_ticket
86,CS,Chưa xác định được vấn đề,47910
92,FIN,Chi hộ vay tiêu dùng,28785
100,FIN,Quốc tế,14150
3,ACC,Hỗ trợ tài khoản,9764
102,FIN,Thu hộ vay tiêu dùng,5709
130,MER,W2B - Chuyển tiền tới ngân hàng,5306
147,PAY,Nạp tiền,4867
25,APP,Hướng dẫn sử dụng ứng dụng,4565
129,MER,W2B - AS - Chuyển tiền tới ngân hàng,3423
111,FIN,Đầu tư tích lũy,2759
